In [2]:
import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\heram\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\heram\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [8]:
words=[]
classes=[]
documents=[]
ignore_words = ['?', '!']
data_file = open(r'C:\Users\heram\OneDrive\Documents\GitHub\Chatbot_scratch\intents.json', encoding='utf-8').read()
intents = json.loads(data_file)

In [9]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [10]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
# Instead of:
# words = sorted(list(set(words)))

# Use:
words = sorted(list(set(words)))

classes=sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)

910 documents
255 classes ['     Support Vector Machine', '   Decision Trees', '  Perceptron', '  Regularization in Machine Learning', '  applications of supervised learning', '  basic data structures', " 'action' in reinforcement learning", " 'agent' in reinforcement learning", " 'value' in reinforcement learning", ' Classification model', ' Gradient Descent', ' Inductive Logic Programming', ' Polynomial Regression', ' Regularization', ' Ridge Regression technique', ' Types of machine learning', ' advantages of using Polynomial Regression over Linear Regression', ' applications of self-supervised learning', ' components of Bayesian logic program', ' difference between heuristic for rule learning and heuristics', ' disadvantages of supervised learning', ' imbalanced datasheet', ' issues to consider in supervised learning', ' standard supervised learning', 'A/B Testing', 'Algorithm techniques in Machine Learning', 'Appreciation', 'Apriori algorithm', 'Areas of Problems', 'BiasML', 'Bina

In [11]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [13]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [ ]:
# max_len = max(len(bag) for bag, _ in training)

# # Pad the bags to the maximum length
# for i in range(len(training)):
#     bag, output_row = training[i]
#     # Pad with 0s at the end
#     training[i][0] = bag + [0] * (max_len - len(bag)) 


In [37]:
# from keras.preprocessing.sequence import pad_sequences

# max_len = max(len(x) for x in training)
# padded_training = pad_sequences(training, maxlen=max_len)

# training = np.array(padded_training)

In [38]:
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents


In [39]:
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax


Training data created


In [41]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0127 - loss: 5.5275
Epoch 2/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0412 - loss: 5.3311
Epoch 3/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0585 - loss: 5.0015
Epoch 4/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1052 - loss: 4.5602
Epoch 5/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1644 - loss: 4.1659
Epoch 6/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2446 - loss: 3.6605
Epoch 7/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - accuracy: 0.2808 - loss: 3.3299
Epoch 8/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2785 - loss: 3.1187
Epoch 9/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3725 - loss: 2.7810
Epoch 10/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4157 - loss: 2.5935
Epoch 11/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4070 - loss: 2.4718
Epoch 12/200
182/182 ━━━━━━━━━━━━━━━━━━

model created


910